## 2. Build regression model on the employee attrition dataset

### Create the 2nd dataframe with 23 variables 
1. Log monthly Income to have same scale
2. Add 4 numerics columns : Cat_Age, Overtime, BusinessTravel, percentSalaryHike


### Apply differents models on the 2st dataframe
1. Logfit 
2. Logistic Regression 
3. Apply Multicollinearity on both models
4. Oversampling : to have the same proportion of Yes/No (45% / 55%)


cf. Google Slides Presentation with all the scores of differents models 

### Test the best model on actual employee that wants to quit and not quit 

In [378]:
import pandas as pd

In [379]:
df=pd.read_csv('EmployeeAttritionv2.csv')

In [380]:
df.Cat_Age=df.Cat_Age.map({'41-50':3, '31-40':2, '18-30':1, '51-60':4})

In [381]:
df.Cat_Age=df.Cat_Age.map({'41-50':3, '31-40':2, '18-30':1, '51-60':4})
df.OverTime=df.OverTime.map({'No':0,'Yes':1}).astype('int64')
df.BusinessTravel=df.BusinessTravel.map({'Non-Travel':0,'Travel_Frequently':2,'Travel_Rarely':1}).astype('int64')
Percent_cat=[1,2,3]
cutoffs = [10,15,20,25]
df['PercentSalaryHike']=pd.cut(df['PercentSalaryHike'],cutoffs, labels=Percent_cat)

In [382]:
df.head()

,Unnamed: 0,Gender,Age,Cat_Age,Attrition,Attrition_bol,Department,JobRole,JobInvolvement,JobLevel,...,RelationshipSatisfaction,NumCompaniesWorked,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,Female,41,NaN,Yes,1,Sales,Sales Executive,3,2,...,1,8,0,8,0,1,6,4,0,5
1,1,Male,49,NaN,No,0,Research & Development,Research Scientist,2,2,...,4,1,1,10,3,3,10,7,1,7
2,2,Male,37,NaN,Yes,1,Research & Development,Laboratory Technician,2,1,...,2,6,0,7,3,3,0,0,0,0
3,3,Female,33,NaN,No,0,Research & Development,Research Scientist,3,1,...,3,1,0,8,3,3,8,7,3,0
4,4,Male,27,NaN,No,0,Research & Development,Laboratory Technician,3,1,...,4,9,1,6,3,3,2,2,2,2


In [383]:
df=pd.get_dummies(df,columns=['Gender'],drop_first=True)

In [384]:
Percent_cat=[1,2,3]
cutoffs = [10,15,20,25]
df['PercentSalaryHike']=pd.cut(df['PercentSalaryHike'],cutoffs, labels=Percent_cat)

In [385]:
df['PercentSalaryHike']=df['PercentSalaryHike'].astype('int64')

In [386]:
df.Gender_Male=df.Gender_Male.astype('int64')


In [387]:
import numpy as np
df.MonthlyIncome=np.log(df.MonthlyIncome)

In [388]:
df.dtypes

Unnamed: 0                    int64
Age                           int64
Cat_Age                     float64
Attrition                    object
Attrition_bol                 int64
Department                   object
JobRole                      object
JobInvolvement                int64
JobLevel                      int64
JobSatisfaction               int64
BusinessTravel                int64
DistanceFromHome              int64
Distance_Home                 int64
MonthlyIncome               float64
PercentSalaryHike             int64
PerformanceRating             int64
OverTime                      int64
Education                     int64
EducationField               object
EnvironmentSatisfaction       int64
MaritalStatus                object
RelationshipSatisfaction      int64
NumCompaniesWorked            int64
StockOptionLevel              int64
TotalWorkingYears             int64
TrainingTimesLastYear         int64
WorkLifeBalance               int64
YearsAtCompany              

In [389]:
df.head()

,Unnamed: 0,Age,Cat_Age,Attrition,Attrition_bol,Department,JobRole,JobInvolvement,JobLevel,JobSatisfaction,...,NumCompaniesWorked,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Gender_Male
0,0,41,NaN,Yes,1,Sales,Sales Executive,3,2,4,...,8,0,8,0,1,6,4,0,5,0
1,1,49,NaN,No,0,Research & Development,Research Scientist,2,2,2,...,1,1,10,3,3,10,7,1,7,1
2,2,37,NaN,Yes,1,Research & Development,Laboratory Technician,2,1,3,...,6,0,7,3,3,0,0,0,0,1
3,3,33,NaN,No,0,Research & Development,Research Scientist,3,1,3,...,1,0,8,3,3,8,7,3,0,0
4,4,27,NaN,No,0,Research & Development,Laboratory Technician,3,1,2,...,9,1,6,3,3,2,2,2,2,1


### Create X and y

In [132]:
y_cat=df.Attrition_bol
X_cat=df.drop(['Attrition_bol','Unnamed: 0','DistanceFromHome','Age'],axis=1)._get_numeric_data()

In [133]:
X_cat.describe().T

,count,mean,std,min,25%,50%,75%,max
Cat_Age,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JobInvolvement,1470.0,2.729932e+00,0.711561,1.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
JobLevel,1470.0,2.063946e+00,1.106940,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00
JobSatisfaction,1470.0,2.728571e+00,1.102846,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00
BusinessTravel,1470.0,1.086395e+00,0.532170,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
Distance_Home,1470.0,1.827891e+00,1.039537,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00
MonthlyIncome,1470.0,8.552515e+00,0.664450,6.916715e+00,7.976252e+00,8.500858e+00,9.033484e+00,9.903438e+00
PercentSalaryHike,1470.0,-9.223372e+18,0.000000,-9.223372e+18,-9.223372e+18,-9.223372e+18,-9.223372e+18,-9.223372e+18
PerformanceRating,1470.0,3.153741e+00,0.360824,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
OverTime,1470.0,2.829932e-01,0.450606,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00


In [134]:
import statsmodels.api as sm

In [135]:
X_cat.shape

(1470, 23)

### Model 5 - Logfit
With different datasets (categories)

In [98]:
#X_cat.drop(['Education','PerformanceRating','PercentSalaryHike','Cat_Age','JobLevel'],axis=1,inplace=True)
X_cat.drop(['Education','PerformanceRating','PercentSalaryHike','Cat_Age','JobLevel','Gender_Male'],axis=1,inplace=True)

In [143]:
X_cat.columns

Index(['Cat_Age', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'BusinessTravel', 'Distance_Home', 'MonthlyIncome', 'PercentSalaryHike',
       'PerformanceRating', 'OverTime', 'Education', 'EnvironmentSatisfaction',
       'RelationshipSatisfaction', 'NumCompaniesWorked', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Gender_Male'],
      dtype='object')

In [146]:
from statsmodels.tools.tools import add_constant
X_cat=add_constant(X_cat)

In [147]:
#X_cat.drop(['Education'],axis=1,inplace=True)
model_5=sm.Logit(y_cat,X_cat)

MissingDataError: exog contains inf or nans

In [148]:
X_cat.shape

(1470, 23)

In [21]:
res_5=model_5.fit()
res_5.summary()

Optimization terminated successfully.
         Current function value: 0.317788
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Attrition_bol   No. Observations:                 1470
Model:                          Logit   Df Residuals:                     1452
Method:                           MLE   Df Model:                           17
Date:                Tue, 26 Nov 2019   Pseudo R-squ.:                  0.2805
Time:                        09:21:54   Log-Likelihood:                -467.15
converged:                       True   LL-Null:                       -649.29
Covariance Type:            nonrobust   LLR p-value:                 5.249e-67
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        8.7175      1.697      5.136      0.000       5.391      12.044
JobInvolvement              -0.5852      0.117     -4.984      0.000      -0.815      -0.355
JobSatisfaction             -0.3828      0.077     -4.976      0.000      -0.534      -0.232
BusinessTravel               0.8781      0.162      5.407      0.000       0.560       1.196
Distance_Home                0.3270      0.079      4.124      0.000       0.172       0.482
MonthlyIncome               -0.7765      0.201     -3.865      0.000      -1.170      -0.383
OverTime                     1.7505      0.178      9.816      0.000       1.401       2.100
EnvironmentSatisfaction     -0.4083      0.078     -5.250      0.000      -0.561      -0.256
RelationshipSatisfaction    -0.2657      0.078     -3.401      0.001      -0.419      -0.113
NumCompaniesWorked           0.1744      0.036      4.871      0.000       0.104       0.245
StockOptionLevel            -0.5949      0.111     -5.344      0.000      -0.813      -0.377
TotalWorkingYears           -0.0875      0.024     -3.718      0.000      -0.134      -0.041
TrainingTimesLastYear       -0.1624      0.069     -2.361      0.018      -0.297      -0.028
WorkLifeBalance             -0.2290      0.115     -1.995      0.046      -0.454      -0.004
YearsAtCompany               0.0859      0.036      2.400      0.016       0.016       0.156
YearsInCurrentRole          -0.1224      0.043     -2.849      0.004      -0.207      -0.038
YearsSinceLastPromotion      0.1593      0.039      4.068      0.000       0.083       0.236
YearsWithCurrManager        -0.1104      0.044     -2.508      0.012      -0.197      -0.024
============================================================================================
"""

In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
X_cat.shape

(1470, 18)

In [25]:
pred_5=round(res_5.predict(X_cat))
confusion_matrix(y_cat,pred_5)

array([[1207,   26],
       [ 145,   92]], dtype=int64)

#### Score_model Logfit

In [28]:
import sklearn.metrics as m
m.recall_score(y_cat,pred_5)

0.3881856540084388

In [29]:
y_cat.value_counts()

0    1233
1     237
Name: Attrition_bol, dtype: int64

### Model 6 - Logistic Regression 

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
? LogisticRegression

In [32]:
model_6=LogisticRegression(class_weight='balanced').fit(X_cat,y_cat)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
pred_6=model_6.predict(X_cat)

In [35]:
confusion_matrix(y_cat,pred_6)

array([[937, 296],
       [ 54, 183]], dtype=int64)

#### Score Modele LogisticRegression

In [38]:
m.recall_score(y_cat,pred_6)

0.7721518987341772

In [39]:
model_6.coef_ 

array([[ 2.59597641, -0.44580097, -0.3196188 ,  0.84155988,  0.28132972,
        -0.29118607,  1.56205407, -0.32776728, -0.17219393,  0.15553555,
        -0.47902223, -0.09390153, -0.13415351, -0.14823178,  0.108073  ,
        -0.12409462,  0.14298147, -0.12864372]])

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = model_6.predict(X_cat)
preds = probs
fpr, tpr, threshold = metrics.roc_curve(y_cat, preds)
roc_auc = metrics.auc(fpr, tpr)
# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Multicollinearity (Model 5 & 6)

In [40]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF

In [41]:
X_cat.drop(['YearsWithCurrManager'],axis=1,inplace=True)

In [42]:
#X_cat.drop(['const','MonthlyIncome','JobInvolvement','WorkLifeBalance','YearsAtCompany'],axis=1,inplace=True)
pd.Series([VIF(X_cat.values,i) for i in range(X_cat.shape[1])],index=X_cat.columns)

const                       377.531712
JobInvolvement                1.006822
JobSatisfaction               1.005632
BusinessTravel                1.006350
Distance_Home                 1.006918
MonthlyIncome                 2.253246
OverTime                      1.020607
EnvironmentSatisfaction       1.008672
RelationshipSatisfaction      1.013940
NumCompaniesWorked            1.230949
StockOptionLevel              1.010663
TotalWorkingYears             3.059702
TrainingTimesLastYear         1.015503
WorkLifeBalance               1.010384
YearsAtCompany                3.707354
YearsInCurrentRole            2.463300
YearsSinceLastPromotion       1.666613
dtype: float64

In [43]:
X_cat.shape
## 2 columns more : Business travel and Overtime

(1470, 17)

### Model 5 - Multicollinearity

In [44]:
X_cat=add_constant(X_cat)
model_5m=sm.Logit(y_cat,X_cat)

res_5m=model_5m.fit()
res_5m.summary()

Optimization terminated successfully.
         Current function value: 0.319883
         Iterations 7


C:\Users\Lenovo\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Attrition_bol   No. Observations:                 1470
Model:                          Logit   Df Residuals:                     1453
Method:                           MLE   Df Model:                           16
Date:                Tue, 26 Nov 2019   Pseudo R-squ.:                  0.2758
Time:                        09:25:44   Log-Likelihood:                -470.23
converged:                       True   LL-Null:                       -649.29
Covariance Type:            nonrobust   LLR p-value:                 2.088e-66
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        8.5502      1.687      5.069      0.000       5.244      11.856
JobInvolvement              -0.6058      0.117     -5.193      0.000      -0.834      -0.377
JobSatisfaction             -0.3753      0.076     -4.920      0.000      -0.525      -0.226
BusinessTravel               0.8775      0.162      5.431      0.000       0.561       1.194
Distance_Home                0.3279      0.079      4.150      0.000       0.173       0.483
MonthlyIncome               -0.7679      0.199     -3.850      0.000      -1.159      -0.377
OverTime                     1.7576      0.178      9.882      0.000       1.409       2.106
EnvironmentSatisfaction     -0.3991      0.077     -5.167      0.000      -0.551      -0.248
RelationshipSatisfaction    -0.2608      0.078     -3.355      0.001      -0.413      -0.108
NumCompaniesWorked           0.1746      0.036      4.895      0.000       0.105       0.245
StockOptionLevel            -0.5887      0.111     -5.321      0.000      -0.806      -0.372
TotalWorkingYears           -0.0870      0.023     -3.716      0.000      -0.133      -0.041
TrainingTimesLastYear       -0.1534      0.069     -2.238      0.025      -0.288      -0.019
WorkLifeBalance             -0.2241      0.114     -1.961      0.050      -0.448      -0.000
YearsAtCompany               0.0454      0.033      1.389      0.165      -0.019       0.109
YearsInCurrentRole          -0.1455      0.042     -3.500      0.000      -0.227      -0.064
YearsSinceLastPromotion      0.1524      0.039      3.931      0.000       0.076       0.228
============================================================================================
"""

In [46]:
pred_5m=round(res_5m.predict(X_cat))
confusion_matrix(y_cat,pred_5m)

array([[1204,   29],
       [ 147,   90]], dtype=int64)

#### Score Model Logit & Multicollinearity

In [48]:
m.recall_score(y_cat,pred_5m)

0.379746835443038

In [50]:
m.precision_score(y_cat,pred_5m)

0.7563025210084033

In [51]:
m.f1_score(y_cat,pred_5m)

0.5056179775280899

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = res_5m.predict(X_cat)
preds = probs
fpr, tpr, threshold = metrics.roc_curve(y_cat, preds)
roc_auc = metrics.auc(fpr, tpr)
# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Modele 6 - LogisticRegression & Multicollinearity

In [52]:
model_6m=LogisticRegression(class_weight='balanced').fit(X_cat,y_cat)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [53]:
pred_6m=model_6m.predict(X_cat)

#### Score Modele 

In [55]:
confusion_matrix(y_cat,pred_6m)

array([[955, 278],
       [ 54, 183]], dtype=int64)

In [56]:
m.recall_score(y_cat,pred_6m)

0.7721518987341772

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = model_6m.predict(X_cat)
preds = probs
fpr, tpr, threshold = metrics.roc_curve(y_cat, preds)
roc_auc = metrics.auc(fpr, tpr)
# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#### Modele Logistic Regression - with different parameters

In [57]:
model=LogisticRegression(class_weight='balanced',verbose=50,solver='liblinear').fit(X_cat,y_cat)

[LibLinear]

In [58]:
pred=model.predict(X_cat)

In [60]:
confusion_matrix(y_cat,pred)

array([[955, 278],
       [ 54, 183]], dtype=int64)

In [62]:
m.recall_score(y_cat,pred)

0.7721518987341772

# Resampling
Modele 7 & 8 with oversampling 

In [63]:
df.shape

(1470, 32)

In [64]:
dfyes=df.loc[df.Attrition=='Yes']

In [65]:
df_sample=dfyes.sample(n=800,random_state=1,replace=True)

In [66]:
df_new=df.append(df_sample)

In [67]:
df_new.shape

(2270, 32)

In [68]:
df_new.Attrition.value_counts()

No     1233
Yes    1037
Name: Attrition, dtype: int64

In [69]:
y_new=df_new.Attrition_bol
X_new=df_new.drop(['Attrition_bol','Unnamed: 0','DistanceFromHome','Age'],axis=1)._get_numeric_data()

In [70]:
X_new.describe().T

,count,mean,std,min,25%,50%,75%,max
Cat_Age,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JobInvolvement,2270.0,2.664317e+00,0.736928,1.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
JobLevel,2270.0,1.934802e+00,1.076278,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,5.000000e+00
JobSatisfaction,2270.0,2.660793e+00,1.111027,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00
BusinessTravel,2270.0,1.147577e+00,0.539151,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
Distance_Home,2270.0,1.921145e+00,1.076387,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00
MonthlyIncome,2270.0,8.455123e+00,0.676229,6.916715e+00,7.884953e+00,8.420352e+00,8.929700e+00,9.903438e+00
PercentSalaryHike,2270.0,-9.223372e+18,0.000000,-9.223372e+18,-9.223372e+18,-9.223372e+18,-9.223372e+18,-9.223372e+18
PerformanceRating,2270.0,3.156388e+00,0.363303,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
OverTime,2270.0,3.757709e-01,0.484428,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00


### Modele 7 Logfit

In [71]:
X_new.drop(['Education','PerformanceRating','PercentSalaryHike','Cat_Age'],axis=1,inplace=True)
#drop[,]
#X_new.drop(['Education','PerformanceRating','PercentSalaryHike','Cat_Age','JobLevel','Gender_Male'],axis=1,inplace=True)

In [72]:
X_new=add_constant(X_new)
model_3=sm.Logit(y_new,X_new)

C:\Users\Lenovo\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [73]:
res_3=model_3.fit()
res_3.summary()

Optimization terminated successfully.
         Current function value: 0.489820
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Attrition_bol   No. Observations:                 2270
Model:                          Logit   Df Residuals:                     2250
Method:                           MLE   Df Model:                           19
Date:                Tue, 26 Nov 2019   Pseudo R-squ.:                  0.2895
Time:                        09:28:11   Log-Likelihood:                -1111.9
converged:                       True   LL-Null:                       -1565.0
Covariance Type:            nonrobust   LLR p-value:                5.481e-180
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       12.8169      1.560      8.216      0.000       9.759      15.875
JobInvolvement              -0.5263      0.075     -7.023      0.000      -0.673      -0.379
JobLevel                     0.5770      0.137      4.203      0.000       0.308       0.846
JobSatisfaction             -0.3573      0.049     -7.328      0.000      -0.453      -0.262
BusinessTravel               0.9048      0.103      8.799      0.000       0.703       1.106
Distance_Home                0.3157      0.050      6.336      0.000       0.218       0.413
MonthlyIncome               -1.2740      0.203     -6.276      0.000      -1.672      -0.876
OverTime                     1.6782      0.113     14.831      0.000       1.456       1.900
EnvironmentSatisfaction     -0.4044      0.048     -8.405      0.000      -0.499      -0.310
RelationshipSatisfaction    -0.2084      0.048     -4.325      0.000      -0.303      -0.114
NumCompaniesWorked           0.1625      0.023      6.982      0.000       0.117       0.208
StockOptionLevel            -0.4742      0.064     -7.388      0.000      -0.600      -0.348
TotalWorkingYears           -0.1069      0.015     -7.212      0.000      -0.136      -0.078
TrainingTimesLastYear       -0.1624      0.042     -3.910      0.000      -0.244      -0.081
WorkLifeBalance             -0.2535      0.071     -3.570      0.000      -0.393      -0.114
YearsAtCompany               0.0914      0.021      4.280      0.000       0.050       0.133
YearsInCurrentRole          -0.0885      0.027     -3.233      0.001      -0.142      -0.035
YearsSinceLastPromotion      0.1457      0.023      6.425      0.000       0.101       0.190
YearsWithCurrManager        -0.0971      0.026     -3.698      0.000      -0.149      -0.046
Gender_Male                  0.2776      0.109      2.551      0.011       0.064       0.491
============================================================================================
"""

In [149]:
X_new.columns

Index(['const', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'BusinessTravel', 'Distance_Home', 'MonthlyIncome', 'OverTime',
       'EnvironmentSatisfaction', 'RelationshipSatisfaction',
       'NumCompaniesWorked', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Gender_Male'],
      dtype='object')

In [150]:
X_cat.columns

Index(['Cat_Age', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'BusinessTravel', 'Distance_Home', 'MonthlyIncome', 'PercentSalaryHike',
       'PerformanceRating', 'OverTime', 'Education', 'EnvironmentSatisfaction',
       'RelationshipSatisfaction', 'NumCompaniesWorked', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Gender_Male'],
      dtype='object')

In [153]:
X_cat.drop(['Cat_Age','PercentSalaryHike','PerformanceRating','Education'],axis=1,inplace=True)

In [156]:
X_cat=add_constant(X_cat)

In [158]:
from sklearn.metrics import confusion_matrix
pred_3=round(res_3.predict(X_cat))
confusion_matrix(y_cat,pred_3)

array([[983, 250],
       [ 61, 176]], dtype=int64)

In [159]:
X_cat.shape

(1470, 20)

In [160]:
X_new.shape

(2270, 20)

#### Score Modele 7 Logfit resamp

In [162]:
import sklearn.metrics as m
m.recall_score(y_cat,pred_3)

0.7426160337552743

### Model 7 Logistic Regression 

In [163]:
from sklearn.linear_model import LogisticRegression

In [164]:
model_4=LogisticRegression(class_weight='balanced').fit(X_new,y_new)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [165]:
pred_4=model_4.predict(X_cat)

In [167]:
confusion_matrix(y_cat,pred_4)

array([[936, 297],
       [ 54, 183]], dtype=int64)

In [169]:
m.recall_score(y_cat,pred_4)

0.7721518987341772

### Multicollinearity  - model 7 & 8

In [170]:
X_new.drop(['const','MonthlyIncome','JobInvolvement','WorkLifeBalance','YearsAtCompany','JobLevel'],axis=1,inplace=True)

In [171]:
#X_new.drop(['const'],axis=1,inplace=True)
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
pd.Series([VIF(X_new.values,i) for i in range(X_new.shape[1])],index=X_new.columns)

JobSatisfaction             5.921947
BusinessTravel              4.966742
Distance_Home               3.732790
OverTime                    1.634729
EnvironmentSatisfaction     5.571457
RelationshipSatisfaction    6.227620
NumCompaniesWorked          2.391903
StockOptionLevel            1.689755
TotalWorkingYears           4.648921
TrainingTimesLastYear       4.852662
YearsInCurrentRole          5.693978
YearsSinceLastPromotion     2.400627
YearsWithCurrManager        5.359695
Gender_Male                 2.455254
dtype: float64

In [172]:
X_new.columns

Index(['JobSatisfaction', 'BusinessTravel', 'Distance_Home', 'OverTime',
       'EnvironmentSatisfaction', 'RelationshipSatisfaction',
       'NumCompaniesWorked', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Gender_Male'],
      dtype='object')

In [173]:
X_new=add_constant(X_new)
model_3m=sm.Logit(y_new,X_new)

C:\Users\Lenovo\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [174]:
res_3m=model_3m.fit()
res_3m.summary()

Optimization terminated successfully.
         Current function value: 0.520416
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Attrition_bol   No. Observations:                 2270
Model:                          Logit   Df Residuals:                     2255
Method:                           MLE   Df Model:                           14
Date:                Tue, 26 Nov 2019   Pseudo R-squ.:                  0.2451
Time:                        19:04:51   Log-Likelihood:                -1181.3
converged:                       True   LL-Null:                       -1565.0
Covariance Type:            nonrobust   LLR p-value:                1.111e-154
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        1.2085      0.307      3.935      0.000       0.607       1.810
JobSatisfaction             -0.3147      0.047     -6.746      0.000      -0.406      -0.223
BusinessTravel               0.8681      0.099      8.789      0.000       0.674       1.062
Distance_Home                0.2887      0.048      6.001      0.000       0.194       0.383
OverTime                     1.5588      0.108     14.411      0.000       1.347       1.771
EnvironmentSatisfaction     -0.3734      0.046     -8.072      0.000      -0.464      -0.283
RelationshipSatisfaction    -0.1935      0.047     -4.156      0.000      -0.285      -0.102
NumCompaniesWorked           0.1217      0.022      5.577      0.000       0.079       0.165
StockOptionLevel            -0.5023      0.062     -8.038      0.000      -0.625      -0.380
TotalWorkingYears           -0.0936      0.010     -9.289      0.000      -0.113      -0.074
TrainingTimesLastYear       -0.1467      0.040     -3.676      0.000      -0.225      -0.068
YearsInCurrentRole          -0.0616      0.024     -2.561      0.010      -0.109      -0.014
YearsSinceLastPromotion      0.1703      0.021      7.943      0.000       0.128       0.212
YearsWithCurrManager        -0.0579      0.024     -2.427      0.015      -0.105      -0.011
Gender_Male                  0.2829      0.105      2.695      0.007       0.077       0.489
============================================================================================
"""

In [175]:
X_cat.drop(['JobLevel','MonthlyIncome','JobInvolvement','WorkLifeBalance','YearsAtCompany'],axis=1,inplace=True)

In [177]:
pred_3m=round(res_3m.predict(X_cat))
confusion_matrix(y_cat,pred_3m)

array([[983, 250],
       [ 62, 175]], dtype=int64)

In [179]:
tn, fp, fn, tp = confusion_matrix(y_cat,pred_3m).ravel()
print(tn, fp, fn, tp)

983 250 62 175


In [180]:
X_new.shape

(2270, 15)

In [181]:
X_cat.shape

(1470, 15)

### Score Model 7 - resamp & multicollinearity

In [183]:
m.recall_score(y_cat,pred_3m)

0.7383966244725738

In [184]:
m.precision_score(y_cat,pred_3m)

0.4117647058823529

In [185]:
m.f1_score(y_cat,pred_3m)

0.5287009063444109

### Modele Logistic Regression 8 -  multicollinearity

In [270]:
model_4m=LogisticRegression(class_weight='balanced').fit(X_new,y_new)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [368]:
pred_4m=model_4m.predict(X_cat.drop('RelationshipSatisfaction',axis=1))

#### Score Model 8m - resamp & Multicollinearity

In [370]:
confusion_matrix(y_cat,pred_4m)

array([[916, 317],
       [ 52, 185]], dtype=int64)

In [242]:
m.precision_score(y_cat,pred_4m)

0.36852589641434264

In [243]:
m.f1_score(y_cat,pred_4m)

0.5006765899864681

In [245]:
m.recall_score(y_cat,pred_4m)

0.7805907172995781

In [256]:
X_new.columns

Index(['const', 'JobSatisfaction', 'BusinessTravel', 'Distance_Home',
       'OverTime', 'EnvironmentSatisfaction', 'NumCompaniesWorked',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Gender_Male'],
      dtype='object')

In [257]:
model_4m.coef_ 

array([[ 0.42384906, -0.30709248,  0.85346647,  0.2779329 ,  1.50065374,
        -0.36327005,  0.11472078, -0.46936084, -0.09090821, -0.14152053,
        -0.05579168,  0.16018699, -0.05478241,  0.24783071]])

In [258]:
data = {'Variable':['const', 'JobSatisfaction', 'BusinessTravel', 'Distance_Home',
       'OverTime', 'EnvironmentSatisfaction', 'NumCompaniesWorked',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Gender_Male'], 'Coefficient':[ 0.42384906, -0.30709248,  0.85346647,  0.2779329 ,  1.50065374,
        -0.36327005,  0.11472078, -0.46936084, -0.09090821, -0.14152053,
        -0.05579168,  0.16018699, -0.05478241,  0.24783071]} 

In [261]:
df=pd.DataFrame(data)
df

,Variable,Coefficient
0,const,0.423849
1,JobSatisfaction,-0.307092
2,BusinessTravel,0.853466
3,Distance_Home,0.277933
4,OverTime,1.500654
5,EnvironmentSatisfaction,-0.363270
6,NumCompaniesWorked,0.114721
7,StockOptionLevel,-0.469361
8,TotalWorkingYears,-0.090908
9,TrainingTimesLastYear,-0.141521


In [194]:
model_4m.intercept_

array([0.63297675])

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = res_3m.predict(X_new)
preds = probs
fpr, tpr, threshold = metrics.roc_curve(y_new, preds)
roc_auc = metrics.auc(fpr, tpr)
# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [195]:
from scipy.special import expit

sigmoid_function = expit(df['x'] * lr.coef_[0][0] + lr.intercept_[0]).ravel()
plt.plot(df[''], sigmoid_function)
plt.scatter(df['X_cat'], df['y'], c=df['y'], cmap='rainbow', edgecolors='b')

KeyError: 'x'

### Select the best model and test it on real employee

In [316]:
des=X_cat.describe().T
des[['min','max']]

,min,max
const,1.0,1.0
JobSatisfaction,1.0,4.0
BusinessTravel,0.0,2.0
Distance_Home,1.0,4.0
OverTime,0.0,1.0
EnvironmentSatisfaction,1.0,4.0
RelationshipSatisfaction,1.0,4.0
NumCompaniesWorked,0.0,9.0
StockOptionLevel,0.0,3.0
TotalWorkingYears,0.0,40.0


In [292]:
columns=['const', 'JobSatisfaction', 'BusinessTravel', 'Distance_Home',
       'OverTime', 'EnvironmentSatisfaction',
       'NumCompaniesWorked', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Gender_Male']

Louis - actually he wants to quit it is current company 

In [347]:
test_Louis=pd.DataFrame({'const':[1], 'JobSatisfaction':[2], 'BusinessTravel':[0], 'Distance_Home':[1],
       'OverTime':[1], 'EnvironmentSatisfaction':[3],
       'NumCompaniesWorked':[1], 'StockOptionLevel':[0], 'TotalWorkingYears':[2],
       'TrainingTimesLastYear':[2], 'YearsInCurrentRole':[3],
       'YearsSinceLastPromotion':[2], 'YearsWithCurrManager':[3], 'Gender_Male':[1]},columns=columns)

In [348]:
pred_test=model_4m.predict(test_Louis)

In [349]:
if pred_test[0] == 1: 
    print('Louis wants to quit Bpifrance! ')
else:
    print('Good news! Louis wants to stay in the company!')

Louis wants to quit Bpifrance! 


Sarah - doesn't want to quit

In [365]:
test_Sarah=pd.DataFrame({'const':[1], 'JobSatisfaction':[3], 'BusinessTravel':[1], 'Distance_Home':[2],
       'OverTime':[0], 'EnvironmentSatisfaction':[3],
       'NumCompaniesWorked':[2], 'StockOptionLevel':[0], 'TotalWorkingYears':[5],
       'TrainingTimesLastYear':[3], 'YearsInCurrentRole':[1],
       'YearsSinceLastPromotion':[1], 'YearsWithCurrManager':[1], 'Gender_Male':[0]}, columns=columns)

In [366]:
pred_test=model_4m.predict(test_Sarah)

In [367]:
if pred_test[0] == 1: 
    print('Sarah wants to quit Bpifrance! ')
else:
    print('Good news! Sarah wants to stay in the company!')

Good news! Sarah wants to stay in the company!
